### Creating and Storing Networks using networkx

This notebook contains some functions that might help in creating and storing networks or graphs.

#### Creating a Graph in networkx

The following code cells show how to convert a distance matrix and a list of nodes into a networkx graph. 

The networkx graph is then saved for later use (for example in streamlit or Gephi).

In [ ]:
#first we have to import networkx       
import networkx as nx

In [ ]:
def cotop_graph_erstellen(nodes, min_weight, distance):
    """
    Create a graph from a list of nodes and a distance matrix
    
    :param nodes: A list of nodes; should be a list of dictionaries, where each dictionary contains certain information about the wanted node
    :param min_weight: acts as a lower bound on the admitted weights of an edge
    :param similarity: a matrix defining the distance between each node. The node id's correspond to the column/row of that node in the matrix
    
    :return: A graph containing nodes and edges between the nodes with different weights assigned to each edge
    """

    # The structure of the graph. Enter the nodes as nodes
    graph = {
        'directed': False,
        'graph': 'semant_graph',
        'links': [],
        'nodes': nodes,
    }

    # To add the edges, we iterate over the different nodes and check the corresponding entry in the distance matrix
    for ix,nodeI in enumerate(graph['nodes']):
        for jx,nodeJ in enumerate(graph['nodes']):
            # As this graph is not directed, we only want to add an edge once. If the graph is directed, we do not need the if condition
            if nodeI['id'] < nodeJ['id']:
                source = nodeI['id']
                target = nodeJ['id']
                weight = distance[ix,jx]
                # If the weight exceeds the minimum weight, we add the edge to our list of edges
                if weight > min_weight:
                    link_dict = {
                        'source':source,
                        'target':target,
                        'weight':weight
                    }
                    graph['links'].append(link_dict)
                    
    return graph

In [ ]:
graph = cotop_graph_erstellen(nodes, 0.0, distance_matrix)

In [ ]:
# The following creates a networkx graph based on the previously defined graph
graph1 = nx.Graph()

# Add the nodes on by one
for node in graph['nodes']:
    # We keep all necessary information about the node. This should be changes according to the wanted graph!!
    graph1.add_node(node['id'],
                    name=node['name'],
                    party=node['party'],
                    msw=node['maxTFIDF'],
                    mfw=node['mfws'],
                    nReden=node['nReden'],
                    nWorte=node['nWorte']);

print('nodes done')

# Add the edges one by one
minWeight = 0.0
for link in graph['links']:
    # If the edge weight exceeds a certain limit, we include it in the networkx graph
    if link['weight'] >= minWeight:
        graph1.add_edge(link['source'], link['target'], weight = link['weight'], community=0)
        
print('links done')

In [ ]:
# this saves the graph for later use
nx.write_gexf(graph1, "graph_v1_0.gexf")
print('save done')

#### Adding a networkx graph to a streamlit webpage

The following code cells showcase how to embedd an interacting graph in a streamlit webpage.

To make it look somewhat good, a lot of fiddeling and some patience is needed. 

To get the basic network visualized however, is rather straightforward.

The following code creates a basic webpage with just the graph. Feel free to adjust the settings according to your specific needs! (To actually make it work, you have to copy it into a seperate Python file)

In [ ]:
# These libraries are needed for the streamlit webpage
import streamlit as st

# These are needed for the interactive network
import streamlit.components.v1 as components
import networkx as nx
from pyvis.network import Network
from IPython.display import HTML

# This sets the webpage basic configuration
st.set_page_config(
    page_title="StreamlitPage",
    layout='centered',
    menu_items={
        'Get Help': 'mailto:abcdefg@gmail.com',
        'Report a bug': "mailto:abcdefg@gmail.com",
        'About': "# This has been created as a first test of what is possible with streamlit."}
)

# This creates a simple headline
st.header("An interactive network with streamlit.")

# This opens the graph in python
G = nx.read_gexf("data/graph_v1_0.gexf")

# This creates the settings for the HTML file which will later contain the graph
graph_ch = Network(height='600px', width='100%', notebook=True, bgcolor="#000000", font_color="white")

# This creates a pyvis graph from the networkx graph (we do this because a pyvis graph is easily stored as a HTML file which we can load into streamlit)
graph_ch.from_nx(G)

# Configure the graph. Play around with the values to see what the settings change
graph_ch.set_options("""
    const options = {
  "nodes": {
    "color": {
         "inherit": true
       },
    "size": {
       "inherit": true
       }
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "selfReferenceSize": null,
    "selfReference": {
      "angle": 0.7853981633974483
    },
    "smooth": false
  },
  "physics": {
    "enabled": true,
    "barnesHut": {
      "gravitationalConstant": -30000
    },
    "minVelocity": 0.075,
    "maxVelocity": 2
  }
}""")

# The pyvis graph is saved in a html file
graph_ch.save_graph("data/semant_parla.html")

# This opens the HTML file
print("preopen")
HtmlFile = open("data/semant_parla.html", 'r', encoding='utf-8')
source_code = HtmlFile.read()

# This embeds the HTML file in the streamlit webpage
print("preshow")
components.html(source_code, height=600, width=700, scrolling=True)